<a href="https://colab.research.google.com/github/born99/Video-Summarization/blob/master/Copy_of_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd
import os
os.chdir('/content/drive/My Drive/Video-Summarization')

!pip install chainer
!pip install functools

/content
     |████████████████████████████████| 1.0 MB 3.7 MB/s 
  Created wheel for chainer: filename=chainer-7.8.1-py3-none-any.whl size=967740 sha256=49bbc00e6af9b60f749764cf4ea094029ba79de69c49742ec6fd26070bf0bebd
  Stored in directory: /root/.cache/pip/wheels/c8/6a/6f/fd563166cc597e5206e375ea074ea836e5db5dd58421215672
Successfully built chainer
  ERROR: Failed building wheel for functools
  Running setup.py clean for functools
Failed to build functools
    Running setup.py install for functools ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-mepkr9mi/functools_7d4e50bf96b24fe7bb9df2daa158dc20/setup.py'"'"'; __file__='"'"'/tmp/pip-install-mepkr9mi/functools_7d4e50bf96b24fe7bb9df2daa158dc20/setup.py'"'"';f = getattr(tokenize, '"'"'open'"'"', open)(__file__) if os.path.exists(__file__) else io.StringIO('"'"'from setuptools import setup; setup()'"'"');code = f.read().replace

In [3]:
!import numpy as np
import os
import json

from func.nets import vid_enc, vid_enc_vgg19
from chainer import serializers

import sys
sys.path.append('script/')
from summarize import get_flabel
from func.sampling.vsum import VSUM

import chainer
from chainer import configuration

/bin/bash: import: command not found


In [4]:
!pwd
# settings
feat_type = 'smt_feat' # smt_feat (proposed) or vgg

# load embedding model
if feat_type == 'smt_feat':
    model = vid_enc.Model()
    serializers.load_npz('data/trained_model/model_par', model)
elif feat_type == 'vgg':
    model = vid_enc_vgg19.Model()
else:
    raise RuntimeError('[invalid feat_type] use smt_feat or vgg')

/content/drive/MyDrive/Video-Summarization


In [5]:
# prepair output dir
d_name = 'summe'
dataset_root = 'data/{}/'.format(d_name)
out_dir = 'results/{:}/{:}/'.format(d_name, feat_type)
print ('save to: ', out_dir)

if not os.path.exists(out_dir):
    os.makedirs(out_dir)

save to:  results/summe/smt_feat/


In [6]:
# load dataset metadata
dataset = json.load(open(dataset_root + 'dataset.json'))
video_id = [d['videoID'] for d in dataset]

print ('Video list:')
for vi in video_id:
    print ('-', vi)

Video list:
- Cooking


In [7]:
# summarize video
v_id = 'Cooking'

with configuration.using_config('train', False):
    with chainer.no_backprop_mode():
        vsum = VSUM(v_id, model, dataset=d_name, seg_l=5)

_, frames, _ = vsum.summarizeRep(seg_l=5, weights=[1.0, 0.0])

# get 0/1 label for each frame
fps = vsum.dataset.data['fps']
fnum = vsum.dataset.data['fnum']
label = get_flabel(frames, fnum, fps, seg_l=5)

loading Cooking
budget:  2
[[ 0.         0.         0.        ...  6.789039   0.         0.955371 ]
 [12.70352    0.         0.        ...  2.812302   0.         0.       ]
 [ 9.723458   0.         0.        ...  0.         0.         6.175164 ]
 ...
 [ 6.3901405  0.         0.        ...  0.         2.5943449  0.       ]
 [ 0.         0.         0.        ...  0.         0.         0.       ]
 [ 2.8705516  0.         0.        ...  0.         0.         0.       ]]
[]


TypeError: ignored

In [ ]:
# write summarized video
from skvideo.io import vread, vwrite

video_path = 'C:/Users/admin/Desktop/vsum_dsf/data/summe/SumMe/videos/%s.webm' % v_id

video_data = vread(video_path)
#sum_vid = video_data[label.ravel().astype(np.bool), :,:,:]

print ('writing video to', 'sum_%s.mp4'%v_id)
vwrite('sum_%s.mp4'%v_id, video_data)